# Test stuff for PFN model

Classify pions, photons, and **scalars**

In [1]:
# Add import paths
import sys
sys.path.append("..")

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import datetime as dt
from utils import print_gpu_memory

# Make tensorflow not use too much memory
import tensorflow as tf
gpus = tf.config.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

disk_dir = "/usatlas/atlas01/atlasdisk/users/atlas_wifeng"

In [2]:
print_gpu_memory()

GPU memory usage
  current: 0B
  peak:    0B


In [3]:
# Get data
from data import get_data
(X_train, X_val, X_test,
 Y_train, Y_val, Y_test) = get_data("scalar_test/all_jets_point_cloud.npz")

In [4]:
# Build model
from model import PFN
Phi_sizes = [128, 128, 128, 128, 64, 64, 64]
F_sizes = [128, 128, 128, 128, 64, 64, 64]

_, n_particles, n_features = X_train.shape
model = PFN(
    n_features=n_features,
    n_particles=n_particles,
    n_outputs=Y_train.shape[1],
    Phi_sizes=Phi_sizes,
    F_sizes=F_sizes
)

In [5]:
from collections import defaultdict
history = defaultdict(list)

In [6]:
# Utility functions for training
from train_model import train_model

def train_iteration(lr, epochs):
    fit_history = train_model(
        model=model, 
        data=(X_train, X_val, Y_train, Y_val),
        lr=lr,
        epochs=epochs
    )
    print(f"\nSaving history...")
    for key in ["loss", "val_loss", "accuracy", "val_accuracy"]:
        history[key].extend(fit_history.history[key])
    return fit_history

def save_model():
    model_dir = f"{disk_dir}/photon-jet/models/pfn"
    cur_date = dt.datetime.now().strftime("%Y-%m-%d")
    os.makedirs(f"{model_dir}/{cur_date}", exist_ok=True)
    model.save(f"{model_dir}/{cur_date}/{dt.datetime.now()}")

In [ ]:
print(f"Training model...")
print(f"=== Training with lr=2e-4 [{dt.datetime.now()}] ===")
train_iteration(lr=1e-3, epochs=10)

Training model...
=== Training with lr=2e-4 [2023-06-07 16:53:23.435861] ===
Epoch 1/10
2400/2400 [==============================] - 62s 19ms/step - loss: 1.2954 - accuracy: 0.3659 - val_loss: 1.0139 - val_accuracy: 0.4627
Epoch 2/10
2400/2400 [==============================] - 45s 19ms/step - loss: 0.9920 - accuracy: 0.4783 - val_loss: 0.9749 - val_accuracy: 0.4887
Epoch 3/10
2400/2400 [==============================] - 45s 19ms/step - loss: 0.9734 - accuracy: 0.4940 - val_loss: 0.9905 - val_accuracy: 0.4784
Epoch 4/10
2400/2400 [==============================] - 45s 19ms/step - loss: 0.9089 - accuracy: 0.5407 - val_loss: 0.7556 - val_accuracy: 0.6306
Epoch 5/10
2400/2400 [==============================] - 45s 19ms/step - loss: 0.7154 - accuracy: 0.6665 - val_loss: 0.5659 - val_accuracy: 0.7450
Epoch 6/10
2400/2400 [==============================] - 45s 19ms/step - loss: 0.4549 - accuracy: 0.8109 - val_loss: 0.4834 - val_accuracy: 0.8025
Epoch 7/10
2400/2400 [=========================